Centro de Informática - CIn/UFPE
Professor:Cleber Zanchettin

PROJETO APRENDIZAGEM DE MÁQUINA

 
Alunos Graduação:
Giulio Carvalho Cavalcante - gcc
Pedro Kempter Brant - pkb

Introdução

O projeto consiste em dar uma resposta para o desafio "Bike Sharing Demand". Um problema de regressão onde se é dado o número de alugueis de bicicleta por hora dos primeiros 20 dias do mês e é pedido uma previsão do número de alugueis para os demais dias de cada mês. Após um pequeno tratamento dos dados e uma análise das variáveis, foi usado o XGBoost Regressor para fazer a previsão.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.special import boxcox, inv_boxcox

train_df=pd.read_csv('../input/train.csv')

train_df.describe()


https://stats.libretexts.org/Bookshelves/Introductory_Statistics/Book%3A_Introductory_Statistics_(Shafer_and_Zhang)/02%3A_Descriptive_Statistics/2.5%3A_The_Empirical_Rule_and_Chebyshev's_Theorem

Chebychev's rule 
Essa regra preve empiricamente que 99% dos dados estão contidos noo intervalo de até 3 desvios padrões da média. Isso pode ser usado para limpar alguns outliers [gráfico 1]. 

In [ ]:
sns.boxplot(train_df['count'])
plt.show()

cnt=train_df['count'].values
q99=np.percentile(cnt,[99])

train_df=train_df[train_df['count']<q99[0]]
sns.distplot(train_df['count'])
plt.show()

https://medium.com/@ODSC/transforming-skewed-data-for-machine-learning-90e6cc364b0
Como pode ser visto no [gráfico 2] acima, isso é um highly skewed data,ou seja, uma distribuição"injusta" que prejudica a classe dos valores menos representados.
3 modelos foram testados para melhorar isso, sendo respectivamente: o boxcox transformation[explicado no link acima], a raiz quadrada, e o log.
As this is a highly skewed data, we will try to transform this data using either log, square-root or box-cox  transformation.
Visualmente, o último(log), deixa a distribuição um pouco mais "balanceada", além de ajudar a diminuir o impacto da diferença nos valores mais altos.

In [ ]:
from scipy.stats import boxcox
sns.distplot(boxcox(train_df['count'])[0])
plt.show()

sns.distplot(train_df['count'].apply(lambda x:np.log(x)))
plt.show()

sns.distplot(train_df['count'].apply(lambda x:x**0.5))
plt.show()
train_df['count']=train_df['count'].apply(lambda x:np.log(x))

Separando o datetime em variáveis possivelmente mais significativas, como hora/dia/mes.

In [ ]:


from datetime import datetime

#converting string dattime to datetime


train_df['datetime']=train_df['datetime'].apply(lambda x:datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))

new_df=train_df

new_df['month']=new_df['datetime'].apply(lambda x:x.month)
new_df['hour']=new_df['datetime'].apply(lambda x:x.hour)

Pode se notar pelo heatmap o grau de correlação entre as variáveis. Partindo das seguintes inferências, foram determinadas quais variáveis descartar:
- Temp(temperatura) e atemp(sensação térmica) são fortemente correlacionadas, atemp será removida para evitar multi-collinearity.
- O número de holidays é muito pequeno para poder extrair alguma informação útil(workinday também será removido).
- A divisão do count em casual ou registered não aparenta ter um comportamento significativamente diferente com a maioria das variáveis.
- O windspeed possui muitos outliers, o que pode prejudicar o desempenho.

In [ ]:
new_df.cov()
sns.heatmap(new_df.corr())
plt.show()

sns.boxplot('windspeed',data=train_df)


In [ ]:
new_df.corr()

In [ ]:
'''
humid + temp + month + hour + season + weather
humid + temp + month + hour
humid + temp + month
humid + temp
'''

final_df=new_df.drop(['datetime', 'holiday', 'workingday', 'atemp', 'windspeed', 'casual', 'registered'], axis=1)

#final_df=new_df.drop(['datetime', 'holiday', 'workingday', 'atemp', 'windspeed', 'casual', 'registered', 'season', 'weather'], axis=1)

#final_df=new_df.drop(['datetime', 'holiday', 'workingday', 'atemp', 'windspeed', 'casual', 'registered', 'season', 'weather', 'hour'], axis=1)

#final_df=new_df.drop(['datetime', 'holiday', 'workingday', 'atemp', 'windspeed', 'casual', 'registered', 'season', 'weather', 'hour', 'month'], axis=1)


final_df.head()




Adding dummy varibles to categorical data

In [ ]:

weather_df=pd.get_dummies(new_df['weather'],prefix='w',drop_first=True)
#year_df=pd.get_dummies(new_df['year'],prefix='y',drop_first=True)
month_df=pd.get_dummies(new_df['month'],prefix='m',drop_first=True)
hour_df=pd.get_dummies(new_df['hour'],prefix='h',drop_first=True)
season_df=pd.get_dummies(new_df['season'],prefix='s',drop_first=True)
                     


final_df=final_df.join(weather_df)
#final_df=final_df.join(year_df)
final_df=final_df.join(month_df)                     
final_df=final_df.join(hour_df)
final_df=final_df.join(season_df)
                     
final_df.head()

'''
#weather_df=pd.get_dummies(new_df['weather'],prefix='w',drop_first=True)
# year_df=pd.get_dummies(new_df['year'],prefix='y',drop_first=True)
month_df=pd.get_dummies(new_df['month'],prefix='m',drop_first=True)
hour_df=pd.get_dummies(new_df['hour'],prefix='h',drop_first=True)
#season_df=pd.get_dummies(new_df['season'],prefix='s',drop_first=True)
                     
#final_df=final_df.join(weather_df)
# final_df=final_df.join(year_df)
final_df=final_df.join(month_df)                     
final_df=final_df.join(hour_df)
#final_df=final_df.join(season_df)
'''

'''
#weather_df=pd.get_dummies(new_df['weather'],prefix='w',drop_first=True)
# year_df=pd.get_dummies(new_df['year'],prefix='y',drop_first=True)
month_df=pd.get_dummies(new_df['month'],prefix='m',drop_first=True)
#hour_df=pd.get_dummies(new_df['hour'],prefix='h',drop_first=True)
#season_df=pd.get_dummies(new_df['season'],prefix='s',drop_first=True)
                     
#final_df=final_df.join(weather_df)
# final_df=final_df.join(year_df)
final_df=final_df.join(month_df)                     
#final_df=final_df.join(hour_df)
#final_df=final_df.join(season_df)
'''

'''
#weather_df=pd.get_dummies(new_df['weather'],prefix='w',drop_first=True)
# year_df=pd.get_dummies(new_df['year'],prefix='y',drop_first=True)
#month_df=pd.get_dummies(new_df['month'],prefix='m',drop_first=True)
#hour_df=pd.get_dummies(new_df['hour'],prefix='h',drop_first=True)
#season_df=pd.get_dummies(new_df['season'],prefix='s',drop_first=True)
                     
#final_df=final_df.join(weather_df)
# final_df=final_df.join(year_df)
#final_df=final_df.join(month_df)                     
#final_df=final_df.join(hour_df)
#final_df=final_df.join(season_df)
'''


In [ ]:

X=final_df.iloc[:,final_df.columns!='count'].values
print (X)

Y=final_df.iloc[:,4].values
# Y=final_df.iloc[:,2].values

print (Y)

https://www.datacamp.com/community/tutorials/xgboost-in-python

O algoritmo escolhido foi o XGBoost(Extreme Gradient Boosting) por ser paralelizável, ter uma ótima performance de tempo, é ser robusto.

Usa o principio de ensemble, onde vários algoritmos mais fracos são combinados(decision trees) para gerar a saida final. Os que tiverem pior performance recebem um peso maior, ou seja, terão maior foco no treinamento nas demais iterações.

Parameters
max_depth (int) – Maximum tree depth for base learners.

min_child_weight (int) – Minimum sum of instance weight(hessian) needed in a child.

gamma (float) – Minimum loss reduction required to make a further partition on a leaf node of the tree.

subsample (float) – Subsample ratio of the training instance.

colsample_bytree (float) – Subsample ratio of columns when constructing each tree.



In [ ]:
import xgboost as xg
xgr=xg.XGBRegressor(max_depth=8,min_child_weight=6,gamma=0.4)
xgr.fit(X,Y)

'''import xgboost as xg
from sklearn.model_selection import GridSearchCV

def grid_search():
    
    xgr=xg.XGBRegressor(max_depth=8,min_child_weight=6,gamma=0.4)
    xgr.fit(X,Y)

    #rf=RandomForestRegressor(n_estimators=100,random_state=0)
    #rf.fit(X,Y)

    #parameters=[{'max_depth':[8,9,10,11,12],'min_child_weight':[4,5,6,7,8]}]
    #parameters=[{'gamma':[i/10.0 for i in range(0,5)]}]
    parameters=[{'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)]}]

    grid_search= GridSearchCV(estimator=xgr, param_grid=parameters, cv=10,n_jobs=-1)

    print (1)
    grid_search=grid_search.fit(X,Y)
    print (2)
    best_accuracy=grid_search.best_score_
    best_parameters=grid_search.best_params_
    print (best_accuracy)
    print (best_parameters)

#if __name__ == '__main__':
   #grid_search()'''

In [ ]:
'''
from sklearn.ensemble import RandomForestRegressor
rf=RandomForestRegressor(n_estimators=100,random_state=0)
rf.fit(X,Y)
imp_list=rf.feature_importances_
feats = {} # a dict to hold feature_name: feature_importance
for feature, importance in zip(final_df.columns, rf.feature_importances_):
    feats[feature] = importance #add the name/value pair
''' 

In [ ]:
new_df.head()

In [ ]:
new_df=pd.read_csv('../input/test.csv')
test_df=pd.read_csv('../input/test.csv')

new_df['datetime']=new_df['datetime'].apply(lambda x:datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))

new_df['month']=new_df['datetime'].apply(lambda x:x.month)
new_df['hour']=new_df['datetime'].apply(lambda x:x.hour)

weather_df=pd.get_dummies(new_df['weather'],prefix='w',drop_first=True)
month_df=pd.get_dummies(new_df['month'],prefix='m',drop_first=True)
hour_df=pd.get_dummies(new_df['hour'],prefix='h',drop_first=True)
season_df=pd.get_dummies(new_df['season'],prefix='s',drop_first=True)


new_df=new_df.join(weather_df)
new_df=new_df.join(month_df)                     
new_df=new_df.join(hour_df)
new_df=new_df.join(season_df)

new_df=new_df.drop(['datetime', 'holiday', 'workingday', 'atemp', 'windspeed'], axis=1)
new_df.head()
                     

In [ ]:
import xgboost as xg
xgr=xg.XGBRegressor(max_depth=8,min_child_weight=6,gamma=0.4,colsample_bytree=0.6,subsample=0.6)
xgr.fit(X,Y)

X_test=new_df.iloc[:,:].values
X_test.shape
#print (new_df.columns)

y_output=xgr.predict(X_test)
y_output

test_df['count'] = pd.Series(np.exp(y_output))
test_df = test_df.drop(['humidity', 'temp', 'season', 'holiday', 'workingday', 'weather', 'atemp', 'windspeed'], axis=1)
test_df.to_csv('sub1.csv', index=False)



Score calculado a partir do erro quadrático médio

Resultados 

humid + temp
Public Score	1.20115

humid + temp + month
Public Score	1.23954

humid + temp + month + hour
Public Score	0.68263

humid + temp + month + hour + season + weather
Public Score	0.65384

Nota-se uma melhora significativa quando a hora é levada em consideração. O resultado final ficou relativamente próximo dos primeiros colocados(~0.3), demonstrando a efetividade do XGBoost.
